In [1]:
import numpy as np
import pandas as pd
import sys
import datetime
import os
from sklearn.metrics.pairwise import manhattan_distances
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import operator  
from functools import reduce

In [2]:
#定义可计算操作时间的用户操作集合
choose='choose'
choose_start='choose_start'
choose_end='choose_end'
connect='connect'
assign='assign'
move='move'
move_start='move_start'
move_end='move_end'
save='SaveProject'
addpou='AddPou'
copy='copy'
paste='paste'
delete='delete'
cut='cut'


behavior_list=[choose,connect,assign,move]

def init_dic(): #初始化可计算操作时间用户操作集合
    result={choose:0,connect:0,assign:0,move:0}
    return result
def behavior2char(data):
    result=''
    if data == choose_start:
        result='A'
    elif data == connect:
        result='B'
    elif data == assign:
        result='C'
    elif data == move_start:
        result='D'
    elif data == save:
        result='E'
    elif data == addpou:
        result='F'
    elif data == copy:
        result='G'
    elif data == paste:
        result='H'
    elif data == delete:
        result='I'
    elif data == cut:
        result='J'
    return result

def behavior2float(data):
    result=0
    if data == choose_start:
        result=1
    elif data == connect:
        result=2
    elif data == assign:
        result=3
    elif data == move_start:
        result=4
    elif data == save:
        result=5
    elif data == addpou:
        result=6
    elif data == copy:
        result=7
    elif data == paste:
        result=8
    elif data == delete:
        result=9
    elif data == cut:
        result=10
    else:
        return
    return result

In [3]:
def pre_data(data): #平台记录代码有bug，这里需要对bug产生对数据进行处理
    n=len(data)
    i=0
    while i<n:
        if(data[i][0]=='connect'):
            if(data[i-1][0]=='move_start'):
                del(data[i-1])
                i=i-1
                n=len(data)
                continue
            if(data[i+1][0]=='move_end'):
                del(data[i+1])
                n=len(data)
                continue
        if(data[i][0]=='assign'):
            if(data[i-1][0]=='move_end' and data[i-2][0]=='move_end' and data[i-3][0]=='move_start'):
                del(data[i-1])
                del(data[i-2])
                del(data[i-3])
                i=i-2
                n=len(data)
                continue
        n=len(data)
        i=i+1
    
    
def generate_time_seq(data): #把数据序列中对时间节点转换成date存入序列
    time_seq=[]
    for tmp in data:
        date_str=tmp[1]+' '+tmp[2]
        date = datetime.datetime.strptime(date_str,'%Y/%m/%d %X')
        time_seq.append(date)
    return time_seq
    
#pre_data(data_seq)
#time_seq=generate_time_seq(data_seq)
#for s in data_seq:
#    print(s)
#for t in time_seq:
#    print(t)

In [4]:
#读入所有用户操作序列
def read_seq(path):#读取单个文件，并分割
    f=open(path)
    data=f.read().splitlines()
    data_seq=[]
    for tmp in data:
        d=tmp.split(' ')
        d.remove('')
        data_seq.append(d)
    return data_seq


def read_all_seq(path):#读取path下面所有不同user目录下的所有文件，并放在user_list中，并生成time_list
    user_list=[]
    user_time_list=[]
    dirs=os.listdir(path)
    for dir in dirs:#所有user目录
        user_tmp=[]
        user_time_tmp=[]
        files=os.listdir(path+dir)
        for file in files:#目录下所有文件
            data=read_seq(path+dir+'/'+file)
            pre_data(data)#读取后对data进行数据清洗
            user_tmp.append(data)
            user_time_tmp.append(generate_time_seq(data))#将生成的时间序列加入
        user_list.append(user_tmp)
        user_time_list.append(user_time_tmp)
    return user_list,user_time_list
user_list,user_time_list=read_all_seq('/home/dope/users/')#user_list就是所有用户所有工程的点击序列，time就是所有用户所有工程的时间序列
print(user_list)
print(user_time_list)

[[[['AddPou', '2018/10/3', '10:31:26'], ['choose_start', '2018/10/3', '10:31:40'], ['choose_end', '2018/10/3', '10:31:43'], ['choose_start', '2018/10/3', '10:31:46'], ['choose_end', '2018/10/3', '10:31:48'], ['choose_start', '2018/10/3', '10:31:50'], ['choose_end', '2018/10/3', '10:31:52'], ['choose_start', '2018/10/3', '10:31:57'], ['choose_end', '2018/10/3', '10:31:59'], ['choose_start', '2018/10/3', '10:32:04'], ['choose_end', '2018/10/3', '10:32:06'], ['choose_start', '2018/10/3', '10:32:10'], ['choose_end', '2018/10/3', '10:32:12'], ['choose_start', '2018/10/3', '10:32:17'], ['choose_end', '2018/10/3', '10:32:19'], ['choose_start', '2018/10/3', '10:32:23'], ['choose_end', '2018/10/3', '10:32:26'], ['choose_start', '2018/10/3', '10:33:01'], ['choose_end', '2018/10/3', '10:33:03'], ['choose_start', '2018/10/3', '10:33:07'], ['choose_end', '2018/10/3', '10:33:08'], ['choose_start', '2018/10/3', '10:33:20'], ['choose_end', '2018/10/3', '10:33:21'], ['choose_start', '2018/10/3', '10:33

#f=open('/Users/iamlira/Downloads/IOTest2.txt') #mac
f=open('/home/dope/users/user_1/IOTest2.txt') #server
data=f.read().splitlines()
data_seq=[]
time_seq=[]
for tmp in data:
    d=tmp.split(' ')
    d.remove('')
    data_seq.append(d)
print(data_seq)

ds1='2018/9/4 21:46:19'
ds2='2018/9/4 21:46:22'
date = datetime.datetime.strptime(ds1,'%Y/%m/%d %X')
date1=datetime.datetime.strptime(ds2,'%Y/%m/%d %X')
print((date1-date).total_seconds())

In [5]:
def pre4lcs(data):
    lcs_list=[]
    for user in data:
        lcs_tmp_list=[]
        for user_behavior in user:
            lcs_str=''
            for tmp in user_behavior:
                lcs_str=lcs_str+behavior2char(tmp[0])
            lcs_tmp_list.append(lcs_str)
        lcs_list.append(lcs_tmp_list)
    return lcs_list


def lcs(a,b):
    lena=len(a)
    lenb=len(b)
    c=[[0 for i in range(lenb+1)] for j in range(lena+1)]
    flag=[[0 for i in range(lenb+1)] for j in range(lena+1)]
    for i in range(lena):
        for j in range(lenb):
            if a[i]==b[j]:
                c[i+1][j+1]=c[i][j]+1
                flag[i+1][j+1]='ok'
            elif c[i+1][j]>c[i][j+1]:
                c[i+1][j+1]=c[i+1][j]
                flag[i+1][j+1]='left'
            else:
                c[i+1][j+1]=c[i][j+1]
                flag[i+1][j+1]='up'
    return flag,c

def printLcs(flag,a,i,j,result):
    if i==0 or j==0:
        return
    if flag[i][j]=='ok':
        printLcs(flag,a,i-1,j-1,result)
        result.append(a[i-1])
        #print(a[i-1],end='')
    elif flag[i][j]=='left':
        printLcs(flag,a,i,j-1,result)
    else:
        printLcs(flag,a,i-1,j,result)
    
def lcs_dis(a,b):
    flag,c = lcs(a,b)
    result=[]
    printLcs(flag,a,len(a),len(b),result)
    return 1-len(''.join(result))/max(len(a),len(b))

a='abcabc'
b='abcabcabcabcabcabcabc'
#for i in c:
#  print(i)
#print('')
#for j in flag:
#  print(j)
print('')
print(lcs_dis(a,b))
print('')


0.7142857142857143



In [6]:
def pre4dtw(data):
    dtw_list=[]
    for user in data:
        dtw_tmp_list=[]
        for user_behavior in user:
            dtw_str=[]
            for tmp in user_behavior:
                temp=behavior2float(tmp[0])
                if temp != None:
                    dtw_str.append(temp)
            dtw_tmp_list.append(dtw_str)
        dtw_list.append(dtw_tmp_list)
    return dtw_list


def dtw(X,Y):
    M=[[manhattan_distances(X[i],Y[j]) for i in range(len(X))] for j in range(len(Y))]
    l1=len(X)
    l2=len(Y) 
    D=[[0 for i in range(l1+1)] for i in range(l2+1)]
    D[0][0]=0 
    for i in range(1,l1+1):
        D[0][i]=sys.maxsize
    for j in range(1,l2+1):
        D[j][0]=sys.maxsize
    for j in range(1,l2+1):
        for i in range(1,l1+1):
            D[j][i]=M[j-1][i-1]+min(D[j-1][i],D[j][i-1],D[j-1][i-1]+M[j-1][i-1])
    return D[-1][-1][-1][-1]/max(len(X),len(Y))

In [7]:
X=[1,2,3,1,2,3]
Y=[1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3]
e=dtw(X,Y)
print(e)

0.444444444444


In [8]:
def pre4edit(data):
    edit_list=[]
    for user in data:
        edit_tmp_list=[]
        for user_behavior in user:
            edit_str=''
            for tmp in user_behavior:
                edit_str=edit_str+behavior2char(tmp[0])
            edit_tmp_list.append(edit_str)
        edit_list.append(edit_tmp_list)
    return edit_list


def edit_distance(str1, str2):
    matrix = [[i+j for j in range(len(str2) + 1)] for i in range(len(str1) + 1)]
    for i in range(1,len(str1)+1):
        for j in range(1,len(str2)+1):
            if str1[i-1] == str2[j-1]:
                d = 0
            else:
                d = 1
            matrix[i][j] = min(matrix[i-1][j]+1,matrix[i][j-1]+1,matrix[i-1][j-1]+d)
    return matrix[len(str1)][len(str2)]/max(len(str1),len(str2))


a='abcabc'
b='abcabcabcabcabcabcabc'
print(edit_distance(a,b))
print(manhattan_distances('1234','12745'))

0.7142857142857143
[[ 11511.]]


In [9]:
def compress_seq(data): #论文中对操作序列进行压缩
    com_seq=[]
    n=len(data)
    i=0
    index = -1
    while i<n: #如果操作序列中连续对操作一样，则合并成一个
        if(len(com_seq)==0 or data[i][0].find(com_seq[index]) == -1):
            if(data[i][0]=='choose_start' or data[i][0]=='choose_end'):
                com_seq.append('choose')
            elif(data[i][0]=='move_start' or data[i][0]=='move_end'):
                com_seq.append('move')
            else:
                com_seq.append(data[i][0])
            index+=1
        i+=1
    return com_seq


def pre4myalgo(data):
    user_list_seq=[]
    for user in data:
        user_tmp=[]
        for seq_list in user:
            user_tmp.append(compress_seq(seq_list))
        user_list_seq.append(user_tmp)
    return user_list_seq


def k_gram(seq,k):
    kgram=set()
    n=len(seq)
    for i in range(n-k+1): #获取连续k个序列的集合
        kgram.add(tuple(seq[i:i+k]))
    return kgram

def Dk(gram_1,gram_2):
    intersection=gram_1.intersection(gram_2)
    union=gram_1.union(gram_2) #取交集，再取并集，1-两者相除
    return 1-(len(intersection)/len(union))

def event_time(data1_seq,data2_seq): #获得在各操作事件上的总时间和
    event_time_set1=init_dic()
    event_time_set2=init_dic()
    n1=len(data1_seq)
    n2=len(data2_seq)
    for i in range(n1):
        if(i+1<n1 and data1_seq[i]=='choose_start' and data1_seq[i+1]=='choose_end'): #获得选择控件的时间和，用end-start
            event_time_set1[choose]=event_time_set1[choose]+data1_seq[i+1][2]-data1_seq[i][2]
        if(i+1<n1 and data1_seq[i]=='move_start' and data1_seq[i+1]=='move_end'): #获得移动控件的时间和，用end-start
            event_time_set1[move]=event_time_set1[move]+data1_seq[i+1][2]-data1_seq[i][2]
        if(i+1<n1 and data1_seq[i]=='assign'): #获取赋值的时间和，用assign操作后的时间值减去assign的
            event_time_set1[assign]=event_time_set1[assign]+data1_seq[i+1][2]-data1_seq[i][2]
        if(i+1<n1 and data1_seq[i]=='connect'): #获取连线的时间和，用connect操作后的时间值减去connect的
            event_time_set1[connect]=event_time_set1[connect]+data1_seq[i+1][2]-data1_seq[i][2]
        
    for i in range(n2):
        if(i+1<n2 and data2_seq[i]=='choose_start' and data2_seq[i+1]=='choose_end'):
            event_time_set2[choose]=event_time_set2[choose]+data2_seq[i+1][2]-data2_seq[i][2]
        if(i+1<n2 and data2_seq[i]=='move_start' and data2_seq[i+1]=='move_end'):
            event_time_set2[move]=event_time_set2[move]+data2_seq[i+1][2]-data2_seq[i][2]
        if(i+1<n2 and data2_seq[i]=='assign'):
            event_time_set2[assign]=event_time_set2[assign]+data2_seq[i+1][2]-data2_seq[i][2]
        if(i+1<n2 and data2_seq[i]=='connect'):
            event_time_set2[connect]=event_time_set2[connect]+data2_seq[i+1][2]-data2_seq[i][2]
            
    event_time_dis=0
    for tmp in behavior_list:
        if(max(event_time_set1[tmp],event_time_set2[tmp]) != 0): #计算两个序列各同个操作事件的事件差
            event_time_dis=event_time_dis+abs(event_time_set1[tmp]-event_time_set2[tmp])/max(event_time_set1[tmp],event_time_set2[tmp])
    event_time_dis=event_time_dis/len(behavior_list) #取平均值
    return event_time_dis

def avg_time_dis(t1_seq,t2_seq): #平均时间差值
    t1_avg=0
    t2_avg=0
    avg_dis=0
    n1=len(t1_seq)
    n2=len(t2_seq)
    for i in range(n1-1):
        t1_avg=t1_avg+(t1_seq[i+1]-t1_seq[i]).total_seconds()
    t1_avg=t1_avg/(n1-1)
    for i in range(n2-1):
        t2_avg=t2_avg+(t2_seq[i+1]-t2_seq[i]).total_seconds()
    t2_avg=t2_avg/(n2-1)
    avg_dis=abs(t1_avg-t2_avg)/max(t1_avg,t2_avg)
    return avg_dis
    

def Dt(data1_seq,t1_seq,data2_seq,t2_seq):
    return avg_time_dis(t1_seq,t2_seq)+event_time(data1_seq,data2_seq)

def my_algo(user_list,user_time_list,k,alpha,beta):#使用论文的算法计算相似度矩阵
    user_com_seq_list = pre4myalgo(user_list)
    user_com_seq_data=reduce(operator.add, user_com_seq_list)
    print(user_com_seq_data)
    user_list_data=reduce(operator.add, user_list)
    user_time_list_data=reduce(operator.add, user_time_list)
    
    n=len(user_com_seq_data)
    simi_matrix=[]
    for i in range(n):
        simi_tmp=[]
        for j in range(n):
            x_gram=k_gram(user_com_seq_data[i],k)
            y_gram=k_gram(user_com_seq_data[j],k)
            Dk_simi = Dk(x_gram,y_gram)
            Dt_simi = Dt(user_list_data[i],user_time_list_data[i],user_list_data[j],user_time_list_data[j])
            simi_tmp.append(alpha*Dk_simi + beta*Dt_simi)
        simi_matrix.append(simi_tmp)
    
    return simi_matrix

a=[2,3,4]
b=[1,2,3,4]
#print(my_algo(a,b,3))
print(my_algo(user_list,user_time_list,3,0.8,0.2))
#print(my_algo(tt[0][0],tt[1][0],3))
#print(Dt(user_list[0][1],user_time_list[0][1],user_list[0][0],user_time_list[0][0]))

[['AddPou', 'choose', 'connect', 'assign', 'SaveProject'], ['AddPou', 'choose', 'connect', 'assign', 'SaveProject'], ['AddPou', 'choose', 'connect', 'assign', 'SaveProject'], ['AddPou', 'choose', 'connect', 'assign', 'SaveProject'], ['AddPou', 'choose', 'move', 'connect', 'SaveProject'], ['AddPou', 'choose', 'move', 'connect', 'SaveProject'], ['AddPou', 'choose', 'move', 'connect', 'SaveProject'], ['AddPou', 'choose', 'move', 'connect', 'SaveProject']]
[[0.0, 0.01030117165998269, 0.011954732510288071, 0.02310879111773966, 0.9163261653327879, 0.9133541963015648, 0.9172639642888472, 0.9171553516381103], [0.01030117165998269, 0.0, 0.0017433537832311022, 0.013503108658953357, 0.9206358580045677, 0.9178169627891385, 0.9215253548204824, 0.921422336357544], [0.011954732510288071, 0.0017433537832311022, 0.0, 0.011863163329076422, 0.9213276568905708, 0.9185333333333334, 0.9222094001682895, 0.9221072796934866], [0.02310879111773966, 0.013503108658953357, 0.011863163329076422, 0.0, 0.925994171169

In [10]:
pre_list=[pre4lcs,pre4dtw,pre4edit]
func_list=[lcs_dis,dtw,edit_distance]


def twotwo_calc(data,func):
    data=reduce(operator.add, data)#把所有序列转换成一维，两两计算序列之间的相似度
    simi_matrix=[]
    n=len(data)
    for i in range(n):
        simi_tmp=[]
        for j in range(n):
            simi_tmp.append(func(data[i],data[j]))
        simi_matrix.append(simi_tmp)
    return simi_matrix

def calc_similarity(data):
    n=len(func_list)
    for i in range(n):
        data_tmp=pre_list[i](data)
        print(twotwo_calc(data_tmp,func_list[i]))

def func(u, v):#直接获取距离矩阵
    v_list = list(v)
    pos_v = v_list.index(0.0)
    return u[pos_v]

def get_cluster_indices(cluster_assignments):
    '''映射每一类至原数据索引
    
    Arguments:
        cluster_assignments 层次聚类后的结果
    
    Returns:
        [[idx1, idx2,..], [idx3]] -- 每一类下的索引
    '''
    n = cluster_assignments.max()
    indices = []
    for cluster_number in range(1, n + 1):
        indices.append(np.where(cluster_assignments == cluster_number)[0])
    
    return indices


def hierarchy_cluster(data,threshold, method='average'):
    '''层次聚类
    
    Arguments:
        data [[0, float, ...], [float, 0, ...]] -- 文档 i 和文档 j 的距离
    
    Keyword Arguments:
        method {str} -- [linkage的方式： single、complete、average、centroid、median、ward] (default: {'average'})
        threshold {float} -- 聚类簇之间的距离
    Return:
        cluster_number int -- 聚类个数
        cluster [[idx1, idx2,..], [idx3]] -- 每一类下的索引
    '''
    data = np.array(data)
 
    Z = linkage(data, method=method , metric = func) #这里的method使用平均数，metric使用自定义的，表示直接从距离矩阵里拿距离
    cluster_assignments = fcluster(Z, t=threshold, criterion='distance')
    print(type(cluster_assignments))
    num_clusters = cluster_assignments.max()
    indices = get_cluster_indices(cluster_assignments)
 
    return num_clusters, indices


In [11]:
calc_similarity(user_list)

[[0.0, 0.5241935483870968, 0.5785714285714285, 0.49572649572649574, 0.2542372881355932, 0.6422764227642277, 0.6834532374100719, 0.6206896551724138], [0.5241935483870968, 0.0, 0.11428571428571432, 0.06451612903225812, 0.6451612903225806, 0.32258064516129037, 0.39568345323741005, 0.33870967741935487], [0.5785714285714285, 0.11428571428571432, 0.0, 0.16428571428571426, 0.6857142857142857, 0.4, 0.30000000000000004, 0.40714285714285714], [0.49572649572649574, 0.06451612903225812, 0.16428571428571426, 0.0, 0.6239316239316239, 0.33333333333333337, 0.40287769784172667, 0.29059829059829057], [0.2542372881355932, 0.6451612903225806, 0.6857142857142857, 0.6239316239316239, 0.0, 0.5284552845528455, 0.5827338129496402, 0.5], [0.6422764227642277, 0.32258064516129037, 0.4, 0.33333333333333337, 0.5284552845528455, 0.0, 0.1151079136690647, 0.06504065040650409], [0.6834532374100719, 0.39568345323741005, 0.30000000000000004, 0.40287769784172667, 0.5827338129496402, 0.1151079136690647, 0.0, 0.165467625899

In [12]:
test=[[0,1,2,3],
      [1,0,4,5],
      [2,4,0,6],
      [3,5,6,0]]

num_clusters, indices = hierarchy_cluster(test,threshold=2.5)

<class 'numpy.ndarray'>


/home/dope/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix


In [13]:
print("%d clusters" % num_clusters)
for k, ind in enumerate(indices):
    print("cluster", k + 1, "is", ind)

3 clusters
cluster 1 is [0 1]
cluster 2 is [2]
cluster 3 is [3]
